In [ ]:
def aspectSegmentationChiSquared(reviews, seeds, vocab=[], threshold=0, iterationLimit=3):
    #when we have the top chi-squared rated keywords, how many do we take
    keywordsToTake = 3
    
    #bootstrap iterations
    
    sent_tokenized_reviews = [nltk.tokenize.sent_tokenize(r.decode('utf-8')) for r in reviews]
    sentences = [sentence for r in sent_tokenized_reviews for sentence in r]
    
    vectorizer = sk.feature_extraction.text.CountVectorizer(min_df = 0, binary=True,
                                                        ngram_range=(1,1),token_pattern = r'[a-zA-Z]{3,}',
                                                        stop_words=nltk.corpus.stopwords.words('english'))
    X = vectorizer.fit_transform(sentences)
    X = X.toarray() #convert sparse array to array
    
    for i in range(0, iterationLimit):
        
        for aspect in seeds:
            seed_indices = [vectorizer.vocabulary_[word] for word in seeds[aspect] if vectorizer.vocabulary_.get(word) != None]
            row_mask = np.logical_or.reduce(X[:,seed_indices].T)
            sentences_with_aspect = X[row_mask,:]
            sentences_without_aspect = X[~row_mask,:]
            c_1 = np.sum(sentences_with_aspect,axis=0)
            c_2 = np.sum(sentences_without_aspect, axis=0)
            c_3 = len(sentences_with_aspect) - c_1
            c_4 = len(sentences_without_aspect) - c_2
            
            numer = 1.0*((c_1*c_4 - c_2 * c_3)**2)
            denom = 1.0*(c_1 + c_3)*(c_2 + c_4)*(c_1 + c_2)*(c_3 + c_4)
            
            csq = numer / denom
            
            count = 0
            sorted_indices = np.argsort(csq)[::-1]
            for x in sorted_indices:
                word = vectorizer.get_feature_names()[x]
                if word not in seeds[aspect]:
                    seeds[aspect].append(word)
                    count += 1
                if count == keywordsToTake:
                    break
                    
    return seeds